In [276]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [277]:
data = pd.read_csv("./data.csv", encoding = 'cp949')
print(data)

# 정상 상태의 데이터만 추출
# 데이터셋에서 결측치 제거
data = data[data['state_3'] == '정상'].reset_index(drop = True)
data

                   측정일시     inlet_3 mw_3 outlet_3 state_3   slurry_3
0       2023-09-01 0:00  189.450000  474     0.68      정상   5.710000
1       2023-09-01 0:05  188.300000  486     0.36      정상   3.070000
2       2023-09-01 0:10  186.300000  510      0.4      정상   3.430000
3       2023-09-01 0:15  188.130000  518     1.74      정상  10.620000
4       2023-09-01 0:20  191.390000  518     1.52      정상  11.990000
...                 ...         ...  ...      ...     ...        ...
51824  2024-02-29 23:35  226.486562  418     5.22      정상  20.185496
51825  2024-02-29 23:40  226.443929  418     4.99      정상  20.213262
51826  2024-02-29 23:45  227.830333  418     4.86      정상  20.188637
51827  2024-02-29 23:50  224.803871  419      4.8      정상  20.196018
51828  2024-02-29 23:55  221.117308  420     5.01      정상  20.196117

[51829 rows x 6 columns]


,측정일시,inlet_3,mw_3,outlet_3,state_3,slurry_3
0,2023-09-01 0:00,189.450000,474,0.68,정상,5.710000
1,2023-09-01 0:05,188.300000,486,0.36,정상,3.070000
2,2023-09-01 0:10,186.300000,510,0.4,정상,3.430000
3,2023-09-01 0:15,188.130000,518,1.74,정상,10.620000
4,2023-09-01 0:20,191.390000,518,1.52,정상,11.990000
...,...,...,...,...,...,...
49341,2024-02-29 23:35,226.486562,418,5.22,정상,20.185496
49342,2024-02-29 23:40,226.443929,418,4.99,정상,20.213262
49343,2024-02-29 23:45,227.830333,418,4.86,정상,20.188637
49344,2024-02-29 23:50,224.803871,419,4.8,정상,20.196018


In [278]:
print(data.dtypes, '\n')

# outlet의 데이터타입을 float형으로 변경
data.outlet_3 = data.outlet_3.astype('float64')
print(data.dtypes)

측정일시         object
inlet_3     float64
mw_3         object
outlet_3     object
state_3      object
slurry_3    float64
dtype: object 

측정일시         object
inlet_3     float64
mw_3         object
outlet_3    float64
state_3      object
slurry_3    float64
dtype: object


In [279]:
# 5분 간격의 데이터를 일별 평균 데이터로 변경
date_pattern = r'(\d{4}-\d{1,2}-\d{1,2})'

data['측정날짜'] = data['측정일시'].str.extract(date_pattern)
groupby_data = data.groupby('측정날짜').mean(numeric_only= True).reset_index()
groupby_data

,측정날짜,inlet_3,outlet_3,slurry_3
0,2023-09-01,178.989477,0.869059,6.545819
1,2023-09-02,177.709686,2.545645,6.832997
2,2023-09-03,185.606969,3.734739,6.989721
3,2023-09-04,200.937317,4.465784,7.451707
4,2023-09-05,243.910976,5.124286,7.925087
...,...,...,...,...
177,2024-02-25,102.011956,20.704635,5.916162
178,2024-02-26,198.172450,4.390245,10.637347
179,2024-02-27,217.054288,5.256014,19.775130
180,2024-02-28,202.838271,5.384630,11.679168


In [280]:
# 필요한 데이터만 남기고 제거
groupby_data = groupby_data[['inlet_3', 'outlet_3', 'slurry_3']]
print(len(groupby_data), '\n')

print(groupby_data.describe())
# inlet, slurry가 음수를 갖는 데이터는 엑셀 상에서 제거
# 이후 describe 결과 상에서 분산이 과도하게 큰 것을 확인, 극단값 5% 정도를 이상치로 상정 후 제거

182 

          inlet_3    outlet_3    slurry_3
count  175.000000  182.000000  175.000000
mean   199.357479   10.695474    7.650087
std     66.647748   28.299923    3.366337
min      0.169584    0.008606    0.013100
25%    173.719243    2.756908    6.446552
50%    200.937317    4.036683    7.406783
75%    250.509929    5.153563    8.815627
max    306.677178  199.505315   28.309601


In [281]:
def detect_outliers(df,columns):
    lower = df[columns].quantile(0.05)
    upper = df[columns].quantile(0.95)
    
    index1= df[df[columns] > upper].index
    index2= df[df[columns] < lower].index 
    
    df[columns] = df[columns].drop(index1)
    df[columns] = df[columns].drop(index2)
    
    return df

detect_outliers(groupby_data, 'inlet_3')
detect_outliers(groupby_data, 'outlet_3')
detect_outliers(groupby_data, 'slurry_3')

,inlet_3,outlet_3,slurry_3
0,178.989477,NaN,6.545819
1,177.709686,2.545645,6.832997
2,185.606969,3.734739,6.989721
3,200.937317,4.465784,7.451707
4,243.910976,5.124286,7.925087
...,...,...,...
177,102.011956,20.704635,5.916162
178,198.172450,4.390245,10.637347
179,217.054288,5.256014,NaN
180,202.838271,5.384630,11.679168


In [282]:
fin_data = groupby_data.dropna().reset_index(drop = True)
print(fin_data)

print(fin_data.describe())

        inlet_3   outlet_3   slurry_3
0    177.709686   2.545645   6.832997
1    185.606969   3.734739   6.989721
2    200.937317   4.465784   7.451707
3    243.910976   5.124286   7.925087
4    161.439728   2.537065   4.953696
..          ...        ...        ...
133  193.068892   2.896923   4.547877
134  237.985851   4.861608   6.278664
135  102.011956  20.704635   5.916162
136  198.172450   4.390245  10.637347
137  202.838271   5.384630  11.679168

[138 rows x 3 columns]
          inlet_3    outlet_3    slurry_3
count  138.000000  138.000000  138.000000
mean   205.359661    4.122380    7.619106
std     43.830801    2.502218    1.829517
min     61.904353    1.530557    1.824824
25%    177.209708    2.834572    6.712495
50%    200.754321    3.837029    7.465243
75%    242.128910    4.748389    8.470375
max    284.497654   20.704635   12.210014
